# Финальный проект по по курсу «Открытые данные» («Статистика»)
*Д. Г. Чесаков, BAE'18*
## Описание сути проекта
Случилось так, что я являюсь игроком в довольно популярную в нашей (и не только) стране игру под названием Dota 2. Это многопользовательская стратегия от третьего лица. Недолго думая, я решил, что будет довольно интересно посчитать что-нибудь связанное с этой игрой. Кратко опишу суть игры -- за каждую команду играет ровно по 5 игроков на 5 разных героях. Все герои в 1 партии различны (то есть в каждой игре участвует ровно 10 разных героев). Так как игра стратегическая -- довольно важны не только сами герои, которые есть у каждой команды (и исполнение на них), но и комбинации героев. Тогда я решил, что будет довольно интересно посмотреть, какие же именно комбинации героев самые победные у каждой отдельной команды (комбинации разные, поскольку каждая команда играет в своем стиле). По итогу я в своем проекте написал функции, которые по названию конкретной команды используют данные ресурса [dotabuff.com](https://www.dotabuff.com), который является крупнейшим сайтом со статистикой по Dota 2, чтобы выгрузить все матчи конкретной команды, героев, которыми они тами играли и вывести лучшие комбинации для каждой из команд, а так же их состав на данный момент.

#### Функция, которая на вход получает название команды, а выдает ее состав.

In [4]:
from bs4 import BeautifulSoup
import requests
from user_agent import generate_user_agent
def team_players(name):
    url = 'https://www.dotabuff.com/esports/teams'
    r = requests.get(url, headers={'User-agent':generate_user_agent()})
    soup = BeautifulSoup(r.text, "html.parser")
    a = soup.findAll('a', class_="esports-link")
    t = ''
    for i in a:
        if name == i.text:
            t = i['href']
            break
    url1 = 'https://www.dotabuff.com/' + t
    r1 = requests.get(url1, headers={'User-agent':generate_user_agent()})
    soup1 = BeautifulSoup(r1.text, "html.parser")
    b = soup1.findAll('a', attrs={"rel":"tooltip-remote"})
    for i in range(5):
        print(b[i].text)
team_players('OG Dota2')  

n0tail
s4
Fly
JerAx
ana


#### Промежуточная функция, которая на вход получает айди матча и команду, а на выход выдает список героев, которыми эта команда играла.

In [31]:
# Эта функция нам нужна, чтобы пройдя по всем матчам команды мы получили их героев из этого матча
def heroes(id_, name):
    url = 'https://www.dotabuff.com' + id_
    r = requests.get(url, headers={'User-agent':generate_user_agent()})
    soup = BeautifulSoup(r.text, "html.parser")
    a = soup.findAll('img', class_="image-hero image-icon image-overlay")
    b = soup.find('header').find('img')
    if b == None:
        b = ''
    else:
        b = b['title']
    # здесь мы смотрим брать нам первых 5 героев или последние 5 в матче (в зависимости от того, перед какими стоит название команды)
    if b == name:
        x = 0
        y = 5
    else:
        x = 5
        y = 10
    z =[]
    if len(a) == 10:
        for i in range(x, y):
            z.append(a[i]['title'])
    return z
heroes('/matches/3084574881', 'Team Empire')

['Ogre Magi', 'Centaur Warrunner', 'Slark', 'Rubick', 'Phantom Assassin']

#### Функция, которая на вход получает адрес страницы и выдает словарь, где каждому айди матча соответствует 0 или 1 (проигрыш или победа нашей команды соответственно).

In [32]:
# Эта функция нам нужна, чтобы вытащить все матчи с 1 страницы
def page_matches(url):
    r = requests.get(url, headers={'User-agent':generate_user_agent()})
    soup = BeautifulSoup(r.text, "html.parser")
    s = {}
    a = soup.findAll('a', class_="won")
    for i in a:
        s[i['href']] = 1
    b = soup.findAll('a', class_="lost")
    for i in b:
        s[i['href']] = 0
    print(s)
page_matches('https://www.dotabuff.com/esports/teams/36/matches?page=4')

{'/matches/2547286015': 1, '/matches/2551023876': 1, '/matches/2551170074': 0, '/matches/2526429133': 1, '/matches/2551241703': 0, '/matches/2548903189': 0, '/matches/2548968400': 0, '/matches/2547115445': 0, '/matches/2526571661': 0, '/matches/2549211234': 1, '/matches/2546943823': 0, '/matches/2547331853': 0, '/matches/2549124641': 1, '/matches/2547165450': 0, '/matches/2547018705': 1, '/matches/2550962469': 1, '/matches/2558534849': 0, '/matches/2526630868': 1, '/matches/2526692434': 1, '/matches/2526759078': 1}


#### Получает на вход название команды, выдает словарь всех матчей этой команды, где выигранному матчу ставится 1, а проигранному 0.

In [36]:
# Эта функция нам нужна, чтобы вытащить уже все матчи (а потом законнектить каждый матч с по его айди с героями)
import re
def all_team_matches(name):
    url = 'https://www.dotabuff.com/esports/teams'
    r = requests.get(url, headers={'User-agent':generate_user_agent()})
    soup = BeautifulSoup(r.text, "html.parser")
    a = soup.findAll('a', class_="esports-link")
    t = ''
    for i in a:
        if name == i.text:
            t = i['href']
            break
    url1 = 'https://www.dotabuff.com/' + t + '/matches?page='
    url_match = 'https://www.dotabuff.com/' + t + '/matches'
    r_match = requests.get(url_match, headers={'User-agent':generate_user_agent()})
    soup_match = BeautifulSoup(r_match.text, "html.parser")
    x = ''
    for i in soup_match.findAll('a'):
        if i.text == 'Last »':
            x = i['href'] 
            break
    # тут мы идем на последнюю страницу с матчами конкретной команды и смотрим в ссылке какой у нее номер, чтобы пройтись по всем
    p = re.compile('=')
    m = p.search(x)
    y = (m.start()) + 1 
    
    x = int(x[y:])
    s = {}
    # ставим в соответствие каждому матчу победа там или поражение
    for i in range(1, x):
        urlx = url1+str(i)
        r2 = requests.get(urlx, headers={'User-agent':generate_user_agent()})
        soup = BeautifulSoup(r2.text, "html.parser")
        a = soup.findAll('a', class_="won")
        for i in a:
            s[i['href']] = 1
        b = soup.findAll('a', class_="lost")
        for i in b:
            s[i['href']] = 0
    return s
all_team_matches('Team NP')

{'/matches/2667057578': 1,
 '/matches/2667103957': 0,
 '/matches/2667169504': 1,
 '/matches/2677586110': 0,
 '/matches/2677624868': 1,
 '/matches/2677667739': 1,
 '/matches/2679837580': 1,
 '/matches/2679884943': 0,
 '/matches/2679987112': 1,
 '/matches/2680032981': 0,
 '/matches/2680122681': 0,
 '/matches/2707751660': 1,
 '/matches/2707859015': 0,
 '/matches/2708158481': 1,
 '/matches/2708211985': 1,
 '/matches/2715490848': 0,
 '/matches/2715554686': 1,
 '/matches/2715610842': 1,
 '/matches/2717502894': 0,
 '/matches/2717560456': 1,
 '/matches/2717600898': 0,
 '/matches/2717653391': 1,
 '/matches/2717713390': 1,
 '/matches/2719307715': 0,
 '/matches/2719395739': 0,
 '/matches/2721020383': 0,
 '/matches/2721116910': 1,
 '/matches/2721261967': 0,
 '/matches/2721319658': 1,
 '/matches/2738480868': 1,
 '/matches/2738673858': 1,
 '/matches/2738812191': 0,
 '/matches/2740497200': 1,
 '/matches/2740785367': 1,
 '/matches/2740960816': 1,
 '/matches/2741056485': 1,
 '/matches/2742937255': 0,
 

#### Программа, которая выдает список списков, где i[5] - победа или поражение команды, а остальные значения - набор героев команды в конкретной игре.

In [35]:
# Тут мы собственно заматчили героев в каждой игре и то, выиграли они или проиграли
import time
name = input()
voc = all_team_matches(name)
def new_voc(voc, name):
    newvoc = []
    for i in voc:
        z = heroes(i, name)
        time.sleep(5)
        if len(z) == 5:
            z.append(voc[i])
            newvoc.append(z)
    return newvoc
b = new_voc(voc, name)
print (b)

Team Liquid
[['Vengeful Spirit', 'Axe', 'Anti-Mage', 'Visage', 'Pugna', 0], ['Dazzle', 'Razor', 'Tusk', 'Earthshaker', 'Huskar', 0], ['Tusk', 'Spectre', "Nature's Prophet", 'Clockwerk', 'Vengeful Spirit', 0], ['Luna', 'Nyx Assassin', 'Disruptor', 'Storm Spirit', 'Venomancer', 1], ['Visage', 'Puck', 'Luna', 'Nyx Assassin', 'Shadow Demon', 1], ['Nyx Assassin', 'Alchemist', 'Slardar', 'Keeper of the Light', 'Lifestealer', 0], ['Luna', "Nature's Prophet", 'Shadow Demon', 'Elder Titan', 'Shadow Shaman', 0], ['Ancient Apparition', 'Dark Seer', 'Doom', 'Templar Assassin', 'Nyx Assassin', 1], ['Gyrocopter', 'Magnus', 'Chen', 'Juggernaut', 'Shadow Demon', 0], ['Treant Protector', 'Tinker', 'Luna', 'Axe', 'Shadow Demon', 0], ['Dark Seer', 'Windranger', 'Dazzle', 'Broodmother', 'Slark', 0], ['Io', 'Razor', 'Witch Doctor', 'Dark Seer', 'Gyrocopter', 1], ['Skywrath Mage', 'Outworld Devourer', 'Tidehunter', 'Razor', 'Shadow Shaman', 0], ['Slardar', 'Morphling', 'Disruptor', 'Dark Seer', 'Dazzle', 0]

#### Возвращет список из всех героев.

In [37]:
# Эта функция нам нужна, чтобы потом составить все возможные пары героев
def all_heroes():
    url = 'https://www.dotabuff.com/heroes'
    r = requests.get(url, headers={'User-agent':generate_user_agent()})
    soup = BeautifulSoup(r.text, "html.parser")
    a = soup.findAll('div', class_="name")
    z = []
    for i in a:
        z.append(i.text)
    return z
all_heroes()

['Abaddon',
 'Alchemist',
 'Ancient Apparition',
 'Anti-Mage',
 'Arc Warden',
 'Axe',
 'Bane',
 'Batrider',
 'Beastmaster',
 'Bloodseeker',
 'Bounty Hunter',
 'Brewmaster',
 'Bristleback',
 'Broodmother',
 'Centaur Warrunner',
 'Chaos Knight',
 'Chen',
 'Clinkz',
 'Clockwerk',
 'Crystal Maiden',
 'Dark Seer',
 'Dazzle',
 'Death Prophet',
 'Disruptor',
 'Doom',
 'Dragon Knight',
 'Drow Ranger',
 'Earth Spirit',
 'Earthshaker',
 'Elder Titan',
 'Ember Spirit',
 'Enchantress',
 'Enigma',
 'Faceless Void',
 'Gyrocopter',
 'Huskar',
 'Invoker',
 'Io',
 'Jakiro',
 'Juggernaut',
 'Keeper of the Light',
 'Kunkka',
 'Legion Commander',
 'Leshrac',
 'Lich',
 'Lifestealer',
 'Lina',
 'Lion',
 'Lone Druid',
 'Luna',
 'Lycan',
 'Magnus',
 'Medusa',
 'Meepo',
 'Mirana',
 'Monkey King',
 'Morphling',
 'Naga Siren',
 "Nature's Prophet",
 'Necrophos',
 'Night Stalker',
 'Nyx Assassin',
 'Ogre Magi',
 'Omniknight',
 'Oracle',
 'Outworld Devourer',
 'Phantom Assassin',
 'Phantom Lancer',
 'Phoenix',
 'Pu

#### По списку матчей выдает табличку из 5 пар героев с наивысшим процентом побед, среди тех пар, которые встречались в не менее, чем 8 матчах (чтобы была какая-то значимость). А так же нынешних игроков команды.

In [38]:
# Это и есть то, зачем делался весь проект, здесь мы получаем искомую табличку + инфу по команде (их состав)
import pandas as pd
import itertools 
name = 'Team NP'
voc = all_team_matches(name)
b = new_voc(voc, name)
z = all_heroes()
x = []
y = []
for i in range(113):
    x.append(float(0))
for i in range(113):
    y.append(x)
df = pd.DataFrame(y, index=z, columns=z)
for i in itertools.combinations(z, r = 2):
    n = 0 
    for j in b:
        if (i[0] in j) and (i[1] in j):
            n += 1
    for j in b:
        if (i[0] in j) and (i[1] in j):
            df.at[i[0], i[1]] = df.at[i[0], i[1]] + j[5]
    if n != 0:
        df.at[i[0], i[1]] = float(df.at[i[0], i[1]]*100/n) 
    if n < 8:
        df.at[i[0], i[1]] = float(0)

t = []        
for i in z:
    for j in z:
        if df.at[i, j] > 0:
            t.append([i, j, df.at[i, j]])
df1 = pd.DataFrame(t)
df1 = df1.sort_values(by = [2], ascending=False)
my_columns = []
my_columns.append('Первый герой')
my_columns.append('Второй герой')
my_columns.append('Процент побед')
df1.columns = my_columns

team_players(name)
print (df1[:5])


MSS
1437
Aui_2000
EternaLEnVy
SVG
  Первый герой Второй герой  Процент побед
0  Beastmaster  Elder Titan      72.727273
3       Mirana      Warlock      66.666667
2      Invoker         Sven      60.000000
1  Beastmaster         Sven      50.000000
